In [6]:
import torch
import gym
from gym import envs
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.distributions import Categorical
from torchvision import transforms
import pickle

In [2]:
lenobs = 21190
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.l1 = nn.Linear(lenobs,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,3)
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)

    def forward(self,x):
        x = F.normalize(x,dim=0)
        y = F.relu(self.l1(x))
        y = F.normalize(y,dim=0)
        y = F.relu(self.l2(y))
        y = F.normalize(y,dim=0)
#         actor = F.softmax(self.actor_lin1(y),dim=0)
        actor = F.log_softmax(self.actor_lin1(y),dim=0)
        c = F.relu(self.l3(y.detach()))
        critic = torch.tanh(self.critic_lin1(c))
        return actor, critic

In [3]:
env = gym.make('Pong-v0')
moveMapping = {
    0:0,
    1:2,
    2:3
}

In [9]:
import time
observation = env.reset()
preprocess = transforms.Compose([
    transforms.ToPILImage(),
])
#Loading model
model = ActorCritic()
model_path = './a2c.pth'
m1 = torch.load(model_path)
model.load_state_dict(m1)
model.eval()


done = False
a = time.time()
while done == False:
    pobservation = torch.from_numpy(observation).permute(2,0,1)
    cropped_image = transforms.functional.crop(preprocess(pobservation),32,15,163,130)
    gs_image = transforms.functional.to_grayscale(cropped_image)
    gs_tensor = transforms.ToTensor()(gs_image)
    flattened_pobservation = gs_tensor.view(-1).float()
    policy, value = model(flattened_pobservation)
    sampler = Categorical(policy)
    action = sampler.sample()
    observation, reward, done, log = env.step(moveMapping[action.item()])
    env.render()
    
if done:
    env.close()
    
b = time.time()-a
print(b)

5.6364781856536865


Number of times agent won

In [8]:
one_reward_count = pickle.load(open('one_reward_count.pkl','rb'))
one_reward_count

[0]